## Alabama 2022 Primary Election Returns

### Georgia 2022 Primary Election Returns

### Sections
- <a href="#ETL">Cleaning Precinct-Level Election Results</a><br>
- <a href="#check">Vote Totals Checks</a><br>
- <a href ="#discrepancies"> Examine/Fix Anomalies <a/><br>
- <a href="#readme">Creating README</a><br>
- <a href="#exp">Exporting Cleaned Precinct-Level Dataset</a><br>

#### Sources

- [Alabama Primary Election Results, Precint Level](https://www.sos.alabama.gov/sites/default/files/election-data/2022-06/2022%20Primary%20Precinct%20Results.zip)
- [Secretary of State Certified Results, Democratic Party](https://www.sos.alabama.gov/sites/default/files/election-2022/AL%20Democratic%20Party%202022%20Primary%20Results.xlsx)
- [Secretary of State Certified Results, Republican Party](https://www.sos.alabama.gov/sites/default/files/election-2022/AL%20Republican%20Party%202022%20Primary%20Results%20Official.xlsx)

In [3]:
import geopandas as gp
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import AL_22_helper_functions as hlp

<p><a name="ETL"></a></p>

### Cleaning Precinct Level Election Returns

Load-In + Clean Election Results

In [4]:
ph_county_list = []
ar_county_list = []
clean_index = []
index_issue = []
files = os.listdir('./raw-from-source/2022 Primary Precinct Results/')

for idx, file in enumerate(files):
    #Load county files
    temp = pd.read_excel('./raw-from-source/2022 Primary Precinct Results/' + file)
    
    # Get the county name, clean "StClair" to match pattern
    county_name = file.split("-")[-1][0:-4]
    if county_name == "StClair":
        county_name = "St. Clair"
    ar_county_list.append(county_name)
        
    # Clean the party name
    temp["Party"] = temp["Party"].str.strip()
    temp["Party"] = temp["Party"].fillna("")
    
    # Create a column to pivot on
    temp["pivot_col"] = temp["Contest Title"].str.strip()+"-:-"+temp["Candidate"].str.strip()
    temp["pivot_col"] = np.where(temp["Party"]=="",temp["pivot_col"],temp["pivot_col"]+"-:-"+temp["Party"].str.strip())
    
    # Drop columns that are no longer needed
    temp.drop(["Contest Title", "Party", "Candidate"], axis = 1, inplace = True)
    
    # Add the county name to the precinct
    rename_dict = {i:i+"-:-"+county_name for i in temp.columns if i != "pivot_col"}
    temp.rename(columns = rename_dict, inplace = True)
    
    # Transpose the dataframe
    temp_transpose = temp.set_index("pivot_col").T
    temp_transpose["County"] = county_name
    temp_transpose.reset_index(inplace = True, drop = False)
    temp_transpose["County"] = county_name
    
    #Make sure cols and indexes unique
    if temp_transpose.columns.nunique() == len(temp_transpose.columns) and temp_transpose.index.is_unique:
        clean_index.append(county_name)
    else:
        index_issue.append(str(county_name) + ' ' + str(idx))
    
#     #Test to make sure all the indexes are unique
#     for val in temp_transpose.index:
#         index_testing.append(val)
#         index_df = pd.DataFrame(index_testing)
        

#     # Add to the list of counties
#     ph_county_list.append(temp_transpose)
    
#     # Concatenate into one file
#     comb = pd.concat(ph_county_list, axis = 0)

In [5]:
len(clean_index)

66

In [6]:
index_issue

['Madison 44']

In [7]:
files[44]

'2022-Primary-Madison.xls'

In [8]:
issue_file = files[44]

In [9]:
temp = pd.read_excel('./raw-from-source/2022 Primary Precinct Results/' + issue_file)

In [15]:
issue_file = files[44]
temp = pd.read_excel('./raw-from-source/2022 Primary Precinct Results/' + issue_file)
    
# Get the county name, clean "StClair" to match pattern
county_name = file.split("-")[-1][0:-4]
ar_county_list.append(county_name)
        
# Clean the party name
temp["Party"] = temp["Party"].str.strip()
temp["Party"] = temp["Party"].fillna("")
    
# Create a column to pivot on
temp["pivot_col"] = temp["Contest Title"].str.strip()+"-:-"+temp["Candidate"].str.strip()
temp["pivot_col"] = np.where(temp["Party"]=="",temp["pivot_col"],temp["pivot_col"]+"-:-"+temp["Party"].str.strip())
    
# # Drop columns that are no longer needed
# temp.drop(["Contest Title", "Party", "Candidate"], axis = 1, inplace = True)
    
# Add the county name to the precinct
rename_dict = {i:i+"-:-"+county_name for i in temp.columns if i != "pivot_col"}
temp.rename(columns = rename_dict, inplace = True)
    
# Transpose the dataframe
temp_transpose = temp.set_index("pivot_col").T
temp_transpose["County"] = county_name
temp_transpose.reset_index(inplace = True, drop = False)
temp_transpose["County"] = county_name
    
#Make sure cols and indexes unique
print(temp_transpose.columns.nunique() == len(temp_transpose.columns))
print(temp_transpose.index.is_unique)
    

False
True


In [39]:
col_dups = pd.DataFrame(temp_transpose.columns.to_list())

In [43]:
col_dups.columns = ['cols']

In [44]:
col_dups['cols'].value_counts()

-:-Over Votes-:-DEM                                                                     4
-:-Under Votes-:-DEM                                                                    4
STATE DEMOCRATIC EXECUTIVE COMMITTEE (MALE), DISTRICT 53-:-Over Votes-:-DEM             1
MADISON COUNTY DEM EXEC COMM (F). DIST 6-:-Laura Hall-:-DEM                             1
STATE REPUBLICAN EXECUTIVE COMMITTEE MADISON COUNTY, PLACE 3-:-Tobias Vogt-:-REP        1
                                                                                       ..
MADISON COUNTY REP EXEC COMM DIST 9 PL 9                TRICT 11-:-Under Votes-:-REP    1
PROPOSED STATEWIDE AMENDMENT NUMBER ONE (1)-:-Yes                                       1
GOVERNOR-:-Under Votes-:-REP                                                            1
GOVERNOR-:-Lew Burdette-:-REP                                                           1
STATE REPUBLICAN EXECUTIVE COMMITTEE MADISON COUNTY, DISTRICT 1-:-Over Votes-:-REP      1
Name: cols

In [16]:
temp_transpose.head()

pivot_col,index,REGISTERED VOTERS - TOTAL-:-Registered Voters - Total,BALLOTS CAST - TOTAL-:-Ballots Cast - Total,BALLOTS CAST - DEMOCRAT-:-Ballots Cast - Alabama Democratic P-:-DEM,BALLOTS CAST - REPUBLICAN-:-Ballots Cast - Alabama Republican P-:-REP,BALLOTS CAST - NON-PARTISAN-:-Ballots Cast - Nonpartisan,BALLOTS CAST - BLANK-:-Ballots Cast - Blank,UNITED STATES SENATOR-:-Will Boyd-:-DEM,UNITED STATES SENATOR-:-Lillie Boddie-:-REP,UNITED STATES SENATOR-:-Brandaun Dean-:-DEM,...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 TRICT 11-:-Elizabeth Black-:-REP,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 TRICT 11-:-Charles Coats-:-REP,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 TRICT 11-:-William Cody Garton-:-REP,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 TRICT 11-:-Over Votes-:-REP,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 TRICT 11-:-Under Votes-:-REP,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 TRICT 11-:-Heath B. Jones-:-REP,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 TRICT 11-:-Stella Stooksbury-:-REP,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 TRICT 11-:-Over Votes-:-REP,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 TRICT 11-:-Under Votes-:-REP,County
0,Contest Title-:-Winston,REGISTERED VOTERS - TOTAL ...,BALLOTS CAST - TOTAL ...,BALLOTS CAST - DEMOCRAT ...,BALLOTS CAST - REPUBLICAN ...,BALLOTS CAST - NON-PARTISAN ...,BALLOTS CAST - BLANK ...,UNITED STATES SENATOR ...,UNITED STATES SENATOR ...,UNITED STATES SENATOR ...,...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 ...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 ...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 ...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 ...,MADISON COUNTY REP EXEC COMM DIST 9 PL 9 ...,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 ...,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 ...,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 ...,MADISON COUNTY REP EXEC COMM DIST 11 PL 2 ...,Winston
1,Party-:-Winston,,,DEM,REP,,,DEM,REP,DEM,...,REP,REP,REP,REP,REP,REP,REP,REP,REP,Winston
2,Candidate-:-Winston,Registered Voters - Total,Ballots Cast - Total,Ballots Cast - Alabama Democratic P,Ballots Cast - Alabama Republican P,Ballots Cast - Nonpartisan,Ballots Cast - Blank,Will Boyd,Lillie Boddie,Brandaun Dean,...,Elizabeth Black,Charles Coats,William Cody Garton,Over Votes,Under Votes,Heath B. Jones,Stella Stooksbury,Over Votes,Under Votes,Winston
3,A & M UNIVERSITY NEW GYM-:-Winston,5742.0,148.0,102.0,46.0,0.0,0.0,70.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winston
4,ABSENTEE-:-Winston,0,1816,356,1456,4,0,211,7,67,...,45,29,21,0,64,19,6,0,20,Winston


In [34]:
temp.head()

,Contest Title-:-Winston,Party-:-Winston,Candidate-:-Winston,A & M UNIVERSITY NEW GYM-:-Winston,ABSENTEE-:-Winston,ALDERSGATE UMC-:-Winston,ALL NATIONS-:-Winston,BOB HARRISON SR_ CTR_-:-Winston,BOBO VFD-:-Winston,BROWNSBORO BAPT CH-:-Winston,...,TONEY UMC-:-Winston,TRIANA CITY HALL-:-Winston,TRINITY UNITED METHODIST-:-Winston,UNIVERSITY BAPT CH-:-Winston,W HUNTSVILLE CH CHRIST-:-Winston,WEATHERLY HGTS BAPT-:-Winston,WEST HUNTSVILLE BAPT-:-Winston,WESTSIDE COMM_ CTR_-:-Winston,WILLOWBROOK BAPT CH-:-Winston,pivot_col
0,REGISTERED VOTERS - TOTAL ...,,Registered Voters - Total,5742.0,0,4962.0,5571.0,3451.0,2659.0,2929.0,...,3623.0,4461.0,6305.0,2309.0,3836.0,4332.0,3467.0,564.0,5980.0,REGISTERED VOTERS - TOTAL-:-Registered Voters ...
1,BALLOTS CAST - TOTAL ...,,Ballots Cast - Total,148.0,1816,1432.0,549.0,629.0,525.0,679.0,...,782.0,666.0,990.0,226.0,513.0,1235.0,269.0,23.0,1803.0,BALLOTS CAST - TOTAL-:-Ballots Cast - Total
2,BALLOTS CAST - DEMOCRAT ...,DEM,Ballots Cast - Alabama Democratic P,102.0,356,136.0,141.0,531.0,59.0,54.0,...,157.0,322.0,126.0,35.0,126.0,117.0,88.0,16.0,150.0,BALLOTS CAST - DEMOCRAT-:-Ballots Cast - Alaba...
3,BALLOTS CAST - REPUBLICAN ...,REP,Ballots Cast - Alabama Republican P,46.0,1456,1295.0,406.0,98.0,466.0,625.0,...,623.0,342.0,863.0,190.0,384.0,1116.0,179.0,7.0,1649.0,BALLOTS CAST - REPUBLICAN-:-Ballots Cast - Ala...
4,BALLOTS CAST - NON-PARTISAN ...,,Ballots Cast - Nonpartisan,0.0,4,1.0,2.0,0.0,0.0,0.0,...,2.0,2.0,1.0,1.0,3.0,2.0,2.0,0.0,4.0,BALLOTS CAST - NON-PARTISAN-:-Ballots Cast - N...


In [31]:
comb_cols = temp[temp.columns[0]].unique()

In [32]:
comb_cols

array(['REGISTERED VOTERS - TOTAL                                             ',
       'BALLOTS CAST - TOTAL                                                  ',
       'BALLOTS CAST - DEMOCRAT                                               ',
       'BALLOTS CAST - REPUBLICAN                                             ',
       'BALLOTS CAST - NON-PARTISAN                                           ',
       'BALLOTS CAST - BLANK                                                  ',
       'UNITED STATES SENATOR                                                 ',
       'UNITED STATES REPRESENTATIVE, 5TH CONGRESSIONAL DISTRICT              ',
       'GOVERNOR                                                              ',
       'ATTORNEY GENERAL                                                      ',
       'ASSOCIATE JUSTICE OF THE SUPREME COURT, PLACE 5                       ',
       'SECRETARY OF STATE                                                    ',
       'STATE AUDITOR       

In [33]:
contest_keywords = ['Senator', 'Representative', 'Governor', 'Attorney General', 'Secretary of State', 'State Auditor', 'State School Superintendent', 'Agriculture', 'Labor', 'Insurance', 'PSC', 'Public Service', 'Supreme Court', 'Court of Appeals', 'Amendment']

In [46]:
comb_list = []
for item in contest_keywords:
    temp = [i for i in comb_cols if item.upper() in i]
    comb_list += temp
    

In [48]:
comb_list

['UNITED STATES SENATOR                                                 ',
 'STATE SENATOR, DISTRICT 2                                             ',
 'UNITED STATES REPRESENTATIVE, 5TH CONGRESSIONAL DISTRICT              ',
 'STATE REPRESENTATIVE, DISTRICT 4                                      ',
 'STATE REPRESENTATIVE, DISTRICT 20                                     ',
 'STATE REPRESENTATIVE, DISTRICT 25                                     ',
 'GOVERNOR                                                              ',
 'ATTORNEY GENERAL                                                      ',
 'SECRETARY OF STATE                                                    ',
 'STATE AUDITOR                                                         ',
 'PUBLIC SERVICE COMMISSION, PLACE 1                                    ',
 'PUBLIC SERVICE COMMISSION, PLACE 2                                    ',
 'ASSOCIATE JUSTICE OF THE SUPREME COURT, PLACE 5                       ',
 'PROPOSED STATEWIDE AMEN

In [28]:

# Filter down to the races we want
comb_list = [i for i in comb.columns if "REPRESENTATIVE" in i] + [i for i in comb.columns if "GOVERNOR" in i] + [i for i in comb.columns if "SENATOR" in i] + [i for i in comb.columns if "ATTORNEY GENERAL" in i] + [i for i in comb.columns if "SECRETARY OF STATE" in i] + [i for i in comb.columns if "TREASURER" in i] + [i for i in comb.columns if "AUDITOR" in i] + [i for i in comb.columns if "AGRICULTURE" in i] + [i for i in comb.columns if "PUBLIC SERVICE" in i] + [i for i in comb.columns if "ASSOCIATE JUSTICE OF THE SUPREME COURT" in i] + [i for i in comb.columns if "CONSTITUTION OF ALABAMA" in i] + [i for i in comb.columns if "PROPOSED STATEWIDE AMENDMENT" in i]

# Remove the under votes and the over votes
comb_list = [i for i in comb_list if "Under Votes" not in i and "Over Votes" not in i]

NameError: name 'comb' is not defined

In [ ]:
ga_22_primary_sw = ga_22_primary[ga_22_primary['contest'].isin(keep_contests)].copy()

<p><a name="check"></a></p>

### Vote Totals Check

<p><a name="discrepancies"></a></p>

### Checks

<p><a name="readme"></a></p>

### Creating README

<p><a name="exp"></a></p>

### Exporting Cleaned Precinct-Level Dataset